## Imports

In [1]:
import datetime
import os
import pickle
import pandas as pd
import s3fs

from melitk.analytics.connectors.core.authentication import Authentication
from melitk.analytics.connectors.teradata import ConnTeradata
from melitk.analytics.connectors.presto import ConnPresto
from melitk.fda import workspace
import datetime
from dateutil.relativedelta import relativedelta
from shared.settings import DATASET_FILENAME, SAMPLE

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from lifetimes.utils import summary_data_from_transaction_data
import numpy as np
import datetime
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from lifetimes import ModifiedBetaGeoFitter
from lifetimes import GammaGammaFitter
import dask.dataframe as dd
from dask.multiprocessing import get
from collections import Counter
from sklearn.model_selection import train_test_split
import gc
import joblib
import boto3
from gensim.models import Doc2Vec
import multiprocessing as mp
from multiprocessing import Process
import gensim
import multiprocessing as mp
from random import shuffle


In [2]:
def _write_dataframe_to_csv_on_s3(df, path_s3):
    import boto3
    from io import StringIO
    """ Write a dataframe to a CSV on S3 """
    a = path_s3.split('//')
    b = a[1].split('/')
    bucket = b[0]
    c = path_s3.split(bucket+'/')
    path = c[1]
    
    buffer = StringIO()
    df.to_csv(buffer,index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, path).put(Body=buffer.getvalue())
    return None

In [ ]:
def print_s3(x,s3_path):
    import boto3
    import datetime
    
    print(x)
    file = s3_path.split('/')[-1]
    with open(file, "a") as myfile:
        myfile.write(str(datetime.date.today().strftime("%m/%d/%Y %H:%M:%S") + ": "+x + "\n"))
        
    s3 = boto3.client('s3')
    s3.upload_file(file, "fda-labs",  '/'.join(s3_path.split('/')[3:])) 

In [3]:
summary = pd.read_csv("s3://fda-labs/ltv-ml/ML/summary_visitas_train.csv",sep = ",")
summary["phrase"] = summary["phrase"].fillna(' ')
summary["phrase"] = summary["phrase"].apply(lambda x: x.split(' '))

In [4]:
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

data = [list(x) for x in summary["phrase"].values]
train_data = list(create_tagged_document(data))

In [ ]:
def model_doc2vec(size,window):
    model = gensim.models.doc2vec.Doc2Vec(vector_size=size, min_count=0, epochs=40,workers = 8,dm = 1,window = window)
    # Build the Volabulary
    model.build_vocab(train_data)
    # Train the Doc2Vec model
    model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)    
    return model


params_grid = [(10,8),(10,12),(10,20),(20,8),(20,12),(20,20),(20,8),(32,12),(32,20),(32,8)]
shuffle(params_grid)

s3 = boto3.client('s3')

for params in params_grid[:4]:
    model = model_doc2vec(*params)
    name = "doc2vec_"+str(params[0])+"_"+str(params[1])
    model.save(name)
    print_s3('||'.join(os.listdir()),"s3://fda-labs/ltv-ml/Embedding/log.txt")
    
    files = [x for x in os.listdir() if(name in x)]
    for f in files:
        s3.upload_file(f, "fda-labs", "ltv-ml/Embedding/"+name+"/"+f) 